In [5]:
import os
import json
import gc
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [1]:
!ls -lha ../data/

total 426M
drwxrwxr-x 3 ec2-user ec2-user 4.0K Jun  8 18:16 .
drwxrwxr-x 6 ec2-user ec2-user 4.0K Jun  6 00:02 ..
drwxrwxr-x 2 ec2-user ec2-user 4.0K Jun  7 23:01 .ipynb_checkpoints
-rw-rw-r-- 1 ec2-user ec2-user 202M Jun  7 23:47 test_clean.csv
-rw-rw-r-- 1 ec2-user ec2-user 224M Jun  7 17:08 train_clean.csv


In [8]:
gc.enable()

features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',\
       'visitNumber', 'visitStartTime', 'device.browser',\
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',\
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',\
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',\
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',\
#        'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',\
       'totals.newVisits', 'totals.pageviews',\
       'trafficSource.adContent', 'trafficSource.campaign',\
       'trafficSource.isTrueDirect', 'trafficSource.keyword',\
       'trafficSource.medium', 'trafficSource.referralPath',\
       'trafficSource.source']

def load_df(csv_path):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
            converters={column: json.loads for column in JSON_COLUMNS}, 
            dtype={'fullVisitorId': 'str'}, # Important!!
            chunksize=100000)
    for df in dfs:
        df.reset_index(drop=True, inplace=True)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        #print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
        use_df = df[features]
        del df
        gc.collect()
        ans = pd.concat([ans, use_df], axis=0).reset_index(drop=True)
        #print(ans.shape)
    return ans

In [9]:
test = load_df('../data/test.csv.zip')

In [ ]:
test.to_csv('../data/test_clean.csv', index = False)

In [1]:
! cat ../data/train\ 2.csv | wc -l

903654
